# __Single-Shell 3-Tissue CSD (SS3T-CSD) modelling analysis notebook__
#### __Last updated on:__ 27/02/2020
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. A bash kernel for IPython (https://github.com/takluyver/bash_kernel)
#### 2. Single-shell DWI dataset (single bvalue)
#### 3. MRtrix3 (https://github.com/MRtrix3/mrtrix3)
#### 4. MRtrix3Tissue (https://3tissue.github.io/)

### __This script includes:__
#### 1. Preprocessing and 3-tissue CSD modelling

### __Define DWI data path__

In [ ]:
#--------------------------------------------------
# Pre-processed DWI data and brain mask 
#--------------------------------------------------
data_path=/home/User/Diffusion
dwi_file=eddy_corrected.nii.gz
brainmask_file=brainmask.nii.gz
bval=dMRI.bvals
bvec=eddy_corrected.eddy_rotated_bvecs

### __Pre-processing__

In [ ]:
# Convert DWI data from NIfTI into mif (MRtrix Image Format) 
mrconvert $data_path/$dwi_file $data_path/dwi.mif -fslgrad $data_path/$bvec $data_path/$bval

# Brain mask
mrconvert $data_path/$brainmask_file $data_path/dwi_mask.mif

# Bias field correction
dwibiascorrect ants $data_path/dwi.mif $data_path/dwi_unbiased.mif

### __Post-processing__

In [ ]:
# 3-tissue response function estimation
dwi2response dhollander $data_path/dwi_unbiased.mif $data_path/response_wm.txt $data_path/response_gm.txt $data_path/response_csf.txt

# Fibre Orientation Distribution estimation (3-tissue CSD modelling)
ss3t_csd_beta1 $data_path/dwi_unbiased.mif $data_path/response_wm.txt $data_path/wmfod.mif $data_path/response_gm.txt \
$data_path/gm.mif $data_path/response_csf.txt $data_path/csf.mif -mask $data_path/dwi_mask.mif

# 3-tissue bias field and intensity normalization
mtnormalise wmfod.mif wmfod_norm.mif gm.mif gm_norm.mif csf.mif csf_norm.mif -mask dwi_mask.mif

### __Visualisation__

In [ ]:
# Compute an FOD-based DEC map from your WM FOD image
fod2dec $data_path/wmfod_norm.mif $data_path/decfod.mif -mask $data_path/dwi_mask.mif
# Overlay the WM FIDs on the FOD-based DEC map
mrview $data_path/decfod.mif -odf.load_sh $data_path/wmfod_norm.mif

### __Tractography__

In [ ]:
# Whole-brain tractography
tckgen data_path/wmfod_norm.mif $data_path/tracks.tck -seed_image $data_path/dwi_mask.mif -select 100000 -cutoff 0.07
# Visualize resulting tractogram
mrview $data_path/decfod.mif -tractography.load $data_path/tracks.tck -tractography.lighting true